In [57]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/venezia/venezia.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_1985.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_1996.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_1993.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_1992.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_1986.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_2009.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_1988.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_2013.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_2005.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_2001.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_1990.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_1984.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_2012.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_1983.csv
/kaggle/input/venezia/Punta_

In [58]:
import os

import numpy as np
import pandas as pd

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import tensorflow.keras.backend as K

In [59]:
config = {
    "N_PREV": 90,
    "N_FWD": 30,
    
    "TEST_SIZE": 0.1
}

In [60]:
df = pd.read_csv('/kaggle/input/venezia/venezia.csv')
df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True)
df.sort_values(by='datetime', inplace=True, ignore_index=True)
df.head()

,datetime,level
0,1983-01-01 01:00:00,44.0
1,1983-01-01 02:00:00,35.0
2,1983-01-01 03:00:00,23.0
3,1983-01-01 04:00:00,10.0
4,1983-01-01 05:00:00,1.0


In [61]:
# values = df.datetime.values.astype('int64') // 10**9
# ranges = values[1:]-values[:-1]
# print(np.unique(ranges, return_counts=True))
# print(np.where(ranges != 3600))
# bad_indecies = [[x, x+1] for x in np.where(ranges!=3600)[0]]
# print(bad_indecies)
# # To make things easy we will drop any samples that have a pair of bad indecies

In [ ]:
# do this stuff locally

In [65]:
data = df.values
tss = TimeSeriesSplit(n_splits=5, test_size=int(df.shape[0]*config["TEST_SIZE"]))
for train_idxs, test_idxs in tss.split(data):
    final_train_sample = train_idxs[-1]
    final_test_sample = test_idxs[-1]
    print(final_train_sample, final_test_sample)
    X_train, y_train = [], []
    for i in range(final_train_sample-config["N_FWD"]-config["N_PREV"]):
        X_train.append(data[i:i+config["N_PREV"]])
        y_train.append(data[i+config["N_PREV"]:i+config["N_PREV"]+config["N_FWD"]])

144636 173563
173563 202490
202490 231417
231417 260344
260344 289271
